In [10]:
from pathlib import Path

# This should match your project root
BASE_DIR = Path.cwd().parent.parent.parent

DATA_RAW = BASE_DIR / "data" / "raw"
DATA_PROCESSED = BASE_DIR / "data" / "processed"

print("BASE_DIR:", BASE_DIR)
print("DATA_RAW exists:", DATA_RAW.exists())
print("DATA_PROCESSED exists:", DATA_PROCESSED.exists())

BASE_DIR: /files/capstone_project/game-market-event-analyzer
DATA_RAW exists: True
DATA_PROCESSED exists: True


In [11]:
import pandas as pd
from pathlib import Path

events = pd.read_csv(DATA_PROCESSED / "events_with_car.csv")
events["trading_date"] = pd.to_datetime(events["trading_date"])
events["event_date"] = pd.to_datetime(events["event_date"])

events.head()


KeyError: 'trading_date'

In [ ]:
prices = pd.read_csv(DATA_PROCESSED / "prices_with_ar.csv")
prices["date"] = pd.to_datetime(prices["date"])

In [ ]:
def compute_car(prices, ticker, trading_date, start, end):
    mask = (
        (prices["ticker"] == ticker) &
        (prices["date"] >= trading_date + pd.Timedelta(days=start)) &
        (prices["date"] <= trading_date + pd.Timedelta(days=end))
    )
    return prices.loc[mask, "AR"].sum()

In [ ]:
events["CAR_0_1"] = events.apply(
    lambda r: compute_car(prices, r["ticker"], r["trading_date"], 0, 1),
    axis=1
)

events["CAR_0_3"] = events.apply(
    lambda r: compute_car(prices, r["ticker"], r["trading_date"], 0, 3),
    axis=1
)

events["CAR_0_5"] = events.apply(
    lambda r: compute_car(prices, r["ticker"], r["trading_date"], 0, 5),
    axis=1
)

In [ ]:
events["CAR_m1_p1"]  # already computed earlier

0     0.005278
1     0.000526
2     0.003371
3     0.001569
4    -0.024080
        ...   
67    0.001016
68    0.018096
69    0.015610
70    0.047224
71    0.003606
Name: CAR_m1_p1, Length: 72, dtype: float64

In [ ]:
# Define ML labels
def label_impact(car):
    if abs(car) > 0.03:
        return "High"
    elif abs(car) > 0.01:
        return "Medium"
    else:
        return "Low"

In [ ]:
events["impact_label"] = events["CAR_m1_p1"].apply(label_impact)
events["impact_label"].value_counts()

impact_label
Low       28
High      27
Medium    17
Name: count, dtype: int64

In [ ]:
out = DATA_PROCESSED / "events_labeled.csv"
events.to_csv(out, index=False)
out

PosixPath('/files/capstone_project/game-market-event-analyzer/data/processed/events_labeled.csv')